In [1]:
import pandas as pd
import csv
data = pd.read_csv("../data/data.csv")

In [2]:
data

,Unnamed: 0.1,Unnamed: 0,product,item_code,store,brand,url,price,item description,image url
0,0,0,Վարդաջուր «Nairian» միցելային,N:001347,Nairian,Nairian,https://4u.am/hy/product/vardajur-nairian-mice...,10500,Հեշտությամբ մաքրեք երկար օրվա հետքերը դեմքից՝ ...,https://static.4u.am/origin/product/1024/00029...
1,1,1,Օծանելիք «Erato Femme Passionnee»,N:018945,Jamharian Perfume,NaN,https://4u.am/hy/product/ocanelik-erato-femme-...,67000,Էրատո թագուհուն նվիրված աֆրոդիզիակ պարունակող ...,https://static.4u.am/origin/product/1024/76nM0...
2,2,2,"Քսուք «Nairian» աչքի, հակատարիքային, երիտասարդ...",N:000696,Nairian,Nairian,https://4u.am/hy/product/acki-ksuk-nairian-hak...,33300,Օգտակար հատկությունները. Բնության զորությամբ լ...,https://static.4u.am/origin/product/1024/00029...
3,3,3,Օծանելիք «Jamharian Collection Blue»,N:018916,Jamharian Perfume,NaN,https://4u.am/hy/product/ocanelik-jamharian-co...,45000,JC Blue-ն գրավում է ցիտրուսային և ծաղկուն նոտա...,https://static.4u.am/origin/product/1024/7kp16...
4,4,4,Քսուք-բալզամ «Բնատուր» աչքերի շուրջ մաշկի համար,N:000466,Bnatur,Bnatur,https://4u.am/hy/product/ksuk-balzam-bnatur-ac...,7700,Սնուցող և հարթեցնող քսուք-բալզամ: \n\n\nՆշանակ...,https://static.4u.am/origin/product/1024/lxjw-...
...,...,...,...,...,...,...,...,...,...,...
252,856,856,Կոմպոզիցիա ''Բատեա'' վարդերով և թրաշուշաններով,N:090517,Flowers Kiosq,NaN,https://4u.am/hy/product/kompozicia-batea-vard...,45000,Ծաղկեփունջը ներառում է՝\n- վարդ\n- պիոնանման վ...,https://static.4u.am/origin/product/1024/4DDgl...
253,857,857,Ծաղկեփունջ «Կորդովա» վարդագույն փնջային վարդերով,N:079749,Bella,NaN,https://4u.am/hy/product/cagkepunj-kordova-var...,56000,Ծաղիկներ՝ փնջային վարդ:\nԵրկարությունը՝ 70 սմ:...,https://static.4u.am/origin/product/1024/txzLY...
254,858,858,Կոմպոզիցիա «Բանարի» փնջային վարդերով և լիմոնիո...,N:090626,Cataleya,NaN,https://4u.am/hy/product/kompozicia-banari-pnj...,17150,"Ծաղիկները՝ փնջային վարդ, լիմոնիում, գիպսոֆիլիա...",https://static.4u.am/origin/product/1024/ticD_...
255,859,859,"Կոմպոզիցիա «Տորոնտո» փնջային վարդերով, քրիզանթ...",N:077320,Bella,NaN,https://4u.am/hy/product/kompozicia-toronto-pn...,30000,"Ծաղիկները՝ փնջային վարդ, վարդ, քրիզանթեմ\n\n\n...",https://static.4u.am/origin/product/1024/spPWh...


In [3]:
product_name = data["product"]
Item_description = data["item description"]




In [18]:
import requests
import cv2
from os import path
import urllib.parse
import numpy as np
from PIL import Image
from io import StringIO
from tensorflow.keras.applications.resnet50 import ResNet50
import os
import tensorflow as tf

In [49]:
for i in data["image url"]:
    img_data = requests.get(i).content
    

    url_parts = urllib.parse.urlparse(i)
    path_parts = url_parts[2].rpartition('/')
    address = r"C:\Users\Admin\Desktop\marketing_analytics_group_project\Recommender_system_4u.am\images"
    imgSaveDir = path.join(address, path_parts[2])
    with open(imgSaveDir, "wb") as handler:
        handler.write(img_data)

In [34]:
lst = []
for image in os.listdir('../images/'):
    i = Image.open('../images/' + image)
    i=np.array(i)
    
    lst.append(i)

In [45]:
lst_= [tf.image.resize(img,size=(224,224)) for img in lst]
model = ResNet50(include_top=True,input_shape=(224, 224, 3))

In [49]:
ar = np.empty((1,1000))
for i in range(len(lst_)):
    m = model(tf.expand_dims(lst_[i],0))
    ar = np.concatenate([ar,m])
with open('arr.npy', 'wb') as f:
    np.save(f,ar)

In [50]:
with open('arr.npy', 'rb') as f:
    ar = np.load(f)

In [52]:
ar.shape

(251, 1000)

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]